In [1]:
import pandas as pd
import numpy as np  
import re
import json
from PIL import Image
import mmcv
import matplotlib.pyplot as plt
from tqdm import tqdm
import os.path as osp
import pathlib
import numpy as np
import shutil


data_root = pathlib.Path('/pub2/data/aging/')
img_dir = '2D_Image'
ann_dir = 'masks'
meta_dir = 'metadata'
remapped_dir = data_root.joinpath('remapped_masks')
if remapped_dir.exists():
    shutil.rmtree(remapped_dir)
remapped_dir.mkdir()

np.random.seed(1)
metadata_reader = pd.read_csv("./aging_individual_item_metadata.csv")

FOOD_CLASSES = metadata_reader.food_item_type.unique().tolist()
PALETTE = np.c_[np.random.choice(range(256), size=(len(FOOD_CLASSES),3), replace=False), 255 + np.zeros(len(FOOD_CLASSES)).astype(int)].tolist()

IMAGE_PATH = data_root.joinpath(img_dir)
ANN_PATH = data_root.joinpath(ann_dir)
META_PATH = data_root.joinpath(meta_dir)

In [6]:
np.array('(140, 255, 25, 255)')

array('(140, 255, 25, 255)', dtype='<U19')

In [16]:
np.array([int(s) for s in re.findall(r'\d+', '(140, 255, 25, 255)')[:-1]]).reshape(1,1,3)

array([[[140, 255,  25]]])

In [53]:
img_sub_dirs = [x for x in IMAGE_PATH.iterdir()]
for img_sub_dir in tqdm(img_sub_dirs):
    if not img_sub_dir.is_dir():
        continue
    # Check scene exists under all circumstances
    scene_dir = img_sub_dir.name
    meta_sub_dir = META_PATH.joinpath(scene_dir)
    ann_sub_dir = ANN_PATH.joinpath(scene_dir)
    if not (meta_sub_dir.exists() and ann_sub_dir.exists()):
        print(f"Warning: {scene_dir} not exists")
        continue
    
    scene_destination = remapped_dir.joinpath(scene_dir)
    if not scene_destination.exists():
        scene_destination.mkdir()
    # Get files in advance to avoid iterate on newly generated files since we will save files as we walk through
    imgs = [x for x in sorted(img_sub_dir.iterdir())]
    anns = [x for x in sorted(ann_sub_dir.iterdir()) if not str(x).endswith('remapped.png')]
    metas = [x for x in sorted(meta_sub_dir.iterdir())]
    for img_file, anno_file, meta_file in zip(imgs, anns, metas):
        # CHECK IF THE SAME FILE
        # pattern_check = "[0-9]+_viewport_[0-9]+"
        # im_re, an_re, me_re = re.search(pattern_check, str(img_file.name)), re.search(pattern_check, str(anno_file.name)), re.search(pattern_check, str(meta_file.name))
        # if not (im_re and an_re and me_re):
        #     raise ValueError("NOT FOUND")
        # if im_re.group() != an_re.group() and im_re.group() != me_re.group():
        #     raise ValueError("NOT FOUND")
        if img_file.name != anno_file.name and img_file.name != meta_file.name:
            raise ValueError("NOT FOUND")

        # Parse mask, retrieve classes and local palette
        metadata_dict = None
        with open(meta_file) as f:
            metadata_dict = json.load(f)
        mask_remap = {}
        for rgb, class_obj in metadata_dict.items():
            cls_name = class_obj['class']
            if cls_name != 'food' and cls_name not in FOOD_CLASSES:
                FOOD_CLASSES.append(cls_name)
                # Generate a new unique palette
                while True:
                    new_color = np.r_[np.random.choice(range(256), size=3), 255].tolist()
                    if new_color not in PALETTE:
                        PALETTE.append(new_color)
                        break
            elif cls_name == 'food':
                cls_name = metadata_reader.loc[metadata_reader['file_name'] == class_obj['food_type'].replace('_','-')].food_item_type.to_list()[0]
            mask_remap[FOOD_CLASSES.index(cls_name)] = [int(s) for s in re.findall(r'\d+', rgb)]
        # print(anno_file)

        # Remap the pixels into our palette
        img = np.array(Image.open(anno_file), dtype=np.uint8)
        seg_map_2d = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        test = [tuple(e) for c in img for e in c]
        # print(test)
        # print("UNIQUE IMAGE PIXEL", set(test))
        for index, color in mask_remap.items():
            m = np.all(img == np.array(color).reshape(1,1,4), axis=2)
            seg_map_2d[m] = index
        # img.save("test.png")
        # print(np.unique(seg_map_2d))
        # np.savetxt('test.txt', seg_map_2d, '%d')
        seg_img = Image.fromarray(seg_map_2d).convert('P')
        seg_img.putpalette(np.array(PALETTE, dtype=np.uint8))
        seg_file_name = remapped_dir.joinpath(scene_dir).joinpath(anno_file.name)
        seg_img.save(seg_file_name)
        # img_arr = np.array(img)
        # img_arr = np.array([[PALETTE[mask_remap['('+ ', '.join(pix.astype(str)) +')']] for pix in row] for row in img_arr]).astype(np.uint8)
        # Image.fromarray(img_arr).save(remapped_dir.joinpath(scene_dir).joinpath(anno_file.name))
        # im = plt.imshow(img_arr)
        # plt.show()
        # break
    # break



100%|██████████| 30/30 [06:17<00:00, 12.60s/it]


In [54]:
assert np.array(PALETTE).shape == np.unique(np.array(PALETTE), axis=0).shape

PALETTE_DICT = {
    class_name: color for color, class_name in zip(PALETTE, FOOD_CLASSES)
}
# print(PALETTE_DICT)
# Serializing json
json_object = json.dumps(PALETTE_DICT, indent=4)
 
# Writing to sample.json
with open("palette_remapped.json", "w") as outfile:
    outfile.write(json_object)